In [ ]:
url = 'https://files.cooperative-robotics.com/intermediate_representation/robotics_knowledge_book_of_future/driving/'








path = '/home/adnan/derp/intermediate_representation/robotics_knowledge_book_of_future/driving'


In [ ]:
path

In [1]:
! uv pip install browser-use

Resolved 125 packages in 925ms                                       
Prepared 76 packages in 1.57s                                            
Uninstalled 5 packages in 33ms
Installed 80 packages in 47msentation-langchain==0.36.1     
 + argparse==1.4.0
 + backoff==2.2.1
 + boto3==1.36.2
 + botocore==1.36.2
 + browser-use==0.1.25
 + cachetools==5.5.0
 + courlan==1.3.2
 + dateparser==1.2.0
 + deprecated==1.2.15
 + filetype==1.2.0
 + fireworks-ai==0.15.11
 + google-ai-generativelanguage==0.6.10
 + google-api-core==2.24.0
 + google-api-python-client==2.159.0
 + google-auth==2.37.0
 + google-auth-httplib2==0.2.0
 + google-generativeai==0.8.3
 + googleapis-common-protos==1.66.0
 + grpcio==1.69.0
 + grpcio-status==1.69.0
 + html2text==2024.2.26
 + htmldate==1.9.3
 + httplib2==0.22.0
 - httpx==0.28.1
 + httpx==0.27.2
 + httpx-sse==0.4.0
 + httpx-ws==0.7.1
 + jmespath==1.0.1
 + justext==3.0.1
 - langchain==0.2.17
 + langchain==0.3.14
 - langchain-anthropic==0.1.23
 + langchain-anthropic==0.3.

In [2]:
!uv pip install langchain chromadb tiktoken  # Install dependencies in Jupyter

Resolved 106 packages in 752ms                                       
Prepared 20 packages in 1.06s                                            
Installed 24 packages in 230ms                              
 + asgiref==3.8.1
 + bcrypt==4.2.1
 + build==1.2.2.post1
 + chroma-hnswlib==0.7.6
 + chromadb==0.6.3
 + coloredlogs==15.0.1
 + durationpy==0.9
 + filelock==3.16.1
 + flatbuffers==24.12.23
 + huggingface-hub==0.27.1
 + humanfriendly==10.0
 + importlib-resources==6.5.2
 + kubernetes==31.0.0
 + mmh3==5.0.1
 + mpmath==1.3.0
 + oauthlib==3.2.2
 + onnxruntime==1.20.1
 + opentelemetry-instrumentation-asgi==0.50b0
 + opentelemetry-instrumentation-fastapi==0.50b0
 + pypika==0.48.9
 + pyproject-hooks==1.2.0
 + requests-oauthlib==2.0.0
 + sympy==1.13.3
 + tokenizers==0.21.0


In [14]:
import os
import subprocess
import json
from typing import List
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# 1) Define your project path
CODEBASE_PATH = '/home/adnan/derp/intermediate_representation/robotics_knowledge_book_of_future/driving'

# 2) Helper function to call `ollama embed` and retrieve the embedding
def get_ollama_embedding(text: str, model: str = "llama2"):
    """
    Calls `ollama embed` via subprocess and returns the embedding as a list of floats.
    Args:
        text: The text to embed
        model: The Ollama model name (default is llama2, or your local model alias)
    """
    # Note: You might need to adjust command parameters based on your model or GPU/CPU usage
    cmd = ["ollama", "embed", "--model", model, text]
    
    # Run Ollama embedding command
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print("Error embedding text:", result.stderr)
        return None
    
    # Parse JSON output to extract embedding array
    try:
        # Ollama's embed output is a JSON object with "embedding": [...]
        output_json = json.loads(result.stdout)
        embedding = output_json.get("embedding", [])
        return embedding
    except json.JSONDecodeError:
        print("Failed to parse Ollama output as JSON.")
        return None


# 3) Build a custom embedding function to integrate with LangChain
#    so it can handle chunks and store them in a vector database.
class OllamaEmbeddingFunction:
    """A LangChain-compatible embedding function that calls `ollama embed`."""
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embeds multiple documents (chunks)."""
        embeddings = []
        for t in texts:
            emb = get_ollama_embedding(t)
            if emb is None:
                emb = []
            embeddings.append(emb)
        return embeddings
    
    def embed_query(self, text: str) -> List[float]:
        """Embeds a single query string."""
        emb = get_ollama_embedding(text)
        return emb if emb else []


# 4) Load files from the directory, split into chunks
def load_and_split_codebase(path: str, suffixes=(".cpp", ".h", ".py", ".java", ".md", ".txt")):
    """
    Recursively load documents from the given path.
    Adjust `suffixes` to match the file types you care about (C++, Python, etc.)
    """
    loader = DirectoryLoader(path, glob="**/*", recursive=True)
    docs = loader.load()
    
    # Filter by suffix if desired (DirectoryLoader might pick up everything)
    # docs = [d for d in docs if any(d.metadata["source"].endswith(s) for s in suffixes)]
    
    # Split large files into smaller chunks to avoid token/embedding limits
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # You can adjust chunk size
        chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(docs)
    return split_docs


# 5) Create or load a vector store with Ollama embeddings
def create_vector_store(documents, embedding_function, store_path="code_index"):
    """
    Creates a Chroma vector store of the given documents using the Ollama embedding function.
    Store path is where the DB will live on disk.
    """
    vectorstore = Chroma.from_documents(
        documents,
        embedding_function,
        persist_directory=store_path
    )
    vectorstore.persist()  # Persist to disk for reuse
    return vectorstore


# 6) Putting it all together: Index the codebase
documents = load_and_split_codebase(CODEBASE_PATH)
ollama_embeddings = OllamaEmbeddingFunction()
code_vector_store = create_vector_store(documents, ollama_embeddings, store_path="driving_code_index")

print(f"Indexed {len(documents)} chunks from the codebase.")
print("Vector store built at 'driving_code_index' directory.")

Error loading file /home/adnan/derp/intermediate_representation/robotics_knowledge_book_of_future/driving/vetur.config.js


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/home/adnan/nltk_data'
    - '/home/adnan/homelab/.venv/nltk_data'
    - '/home/adnan/homelab/.venv/share/nltk_data'
    - '/home/adnan/homelab/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [9]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /home/adnan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
! uv pip install unstructured

Resolved 56 packages in 447ms                                        
Prepared 18 packages in 662ms                                            
Installed 19 packages in 10ms                               
 + aiofiles==24.1.0
 + chardet==5.2.0
 + cryptography==44.0.0
 + emoji==2.14.1
 + eval-type-backport==0.2.2
 + html5lib==1.1
 + joblib==1.4.2
 + jsonpath-python==1.0.6
 + langdetect==1.0.9
 + ndjson==0.3.1
 + nltk==3.9.1
 + olefile==0.47
 + pypdf==5.1.0
 + python-iso639==2024.10.22
 + python-magic==0.4.27
 + python-oxmsg==0.0.1
 + rapidfuzz==3.11.0
 + unstructured==0.16.14
 + unstructured-client==0.29.0


In [10]:
from langchain.vectorstores import Chroma
code_vector_store = Chroma(
    embedding_function=ollama_embeddings,
    persist_directory="driving_code_index"
)

NameError: name 'ollama_embeddings' is not defined